In [2]:
# lookup https://docs.flexcompute.com/projects/photonforge/en/v1.1.2/examples/Y_Splitter.html

import numpy as np
import photonforge as pf
import photonforge.live_viewer as live_viewer
from matplotlib import pyplot as plt

viewer = live_viewer.LiveViewer(port=5002)

Starting live viewer at http://localhost:5002


In [3]:
tech = pf.basic_technology()
pf.config.default_technology = tech

In [4]:
tech.layers

Name,Layer,Description,Color,Pattern
WG_CLAD,"(1, 0)",Waveguide clad,#9da6a218,.
WG_CORE,"(2, 0)",Waveguide core,#6db5dd18,/
SLAB,"(3, 0)",Slab region,#8851ad18,:
TRENCH,"(4, 0)",Deep-etched trench for chip…… facets,#535e5918,+
METAL,"(5, 0)",Metal layer,#b8a18b18,\


In [5]:
print("Port specifications:")
for name, spec in tech.ports.items():
    print(f"- {name}: {spec!r}")

Port specifications:
- Strip: PortSpec(description="Strip waveguide", width=2.25, limits=(-1, 1.22), num_modes=1, added_solver_modes=0, polarization="", target_neff=4, default_radius=0, path_profiles=[(0.5, 0, (2, 0)), (2.5, 0, (1, 0))])
- Rib: PortSpec(description="Rib waveguide", width=2.16, limits=(-1, 1.22), num_modes=1, added_solver_modes=0, polarization="", target_neff=4, default_radius=0, path_profiles=[(0.4, 0, (2, 0)), (2.4, 0, (3, 0)), (2.4, 0, (1, 0))])
- CPW: PortSpec(description="CPW transmission line", width=26.0552, limits=(-9.05312, 12.9931), num_modes=1, added_solver_modes=1, polarization="", target_neff=4, default_radius=0, path_profiles={"signal": (3.6, 0, (5, 0)), "gnd0": (8.9776, -8.7888, (5, 0)), "gnd1": (8.9776, 8.7888, (5, 0))}, voltage_path=[(4.3, 1.97), (1.8, 1.97)], current_path=[(3.05, 3.47), (-3.05, 3.47), (-3.05, 0.47), (3.05, 0.47)])


In [6]:
print("Extrusion specifications:")
for i, spec in enumerate(tech.extrusion_specs):
    print(f"{i + 1}. {spec!r}")

Extrusion specifications:
1. ExtrusionSpec(mask_spec=MaskSpec((2, 0)), medium={"optical": cSi_Li1993_293K, "electrical": Medium(permittivity=12.3)}, limits=(0, 0.22), sidewall_angle=0, reference="top")
2. ExtrusionSpec(mask_spec=MaskSpec((3, 0)), medium={"optical": cSi_Li1993_293K, "electrical": Medium(permittivity=12.3)}, limits=(0, 0.07), sidewall_angle=0, reference="top")
3. ExtrusionSpec(mask_spec=MaskSpec((5, 0)), medium={"optical": Cu_JohnsonChristy1972, "electrical": PEC}, limits=(1.72, 2.22), sidewall_angle=0, reference="top")
4. ExtrusionSpec(mask_spec=MaskSpec((4, 0)), medium={"optical": Medium(), "electrical": Medium()}, limits=(-2e+07, 2e+07), sidewall_angle=0, reference="top")


In [ ]:
# Values from the optimized Y splitter
w = np.array((0.5, 0.5, 0.6, 0.7, 0.9, 1.26, 1.4, 1.4, 1.4, 1.4, 1.31, 1.2, 1.2))
w_reversed = w[::-1]
length = 2.0
y_out = 0.35

vertices = np.vstack((np.linspace(0, length, len(w)), -0.5 * w)).T
vertices_reversed = np.vstack((np.linspace(0, length, len(w_reversed)), -0.5 * w_reversed)).T
y_polygon = pf.Polygon(np.vstack((vertices, vertices[::-1] * np.array((1, -1)))))
y_polygon_combiner = pf.Polygon(np.vstack((vertices_reversed, vertices_reversed[::-1] * np.array((1, -1)))))

y_polygon #Splitter
y_polygon_combiner #Combiner

In [7]:
port_spec = pf.config.default_technology.ports["Strip"]

bimodal_clad_margin = 2 * (w[-1] + 2)
bimodal_port = pf.PortSpec(
    "Output test",
    width=0.9 * bimodal_clad_margin,
    limits=port_spec.limits,
    num_modes=2 * port_spec.num_modes,
    target_neff=port_spec.target_neff,
    path_profiles={
        (bimodal_clad_margin, 0, "WG_CLAD"),
        (0.5, y_out, "WG_CORE"),
        (0.5, -y_out, "WG_CORE"),
    },
)

In [9]:
clad_width, _ = port_spec.path_profile_for("WG_CLAD")
core_width, _ = port_spec.path_profile_for("WG_CORE")
clad_margin = (clad_width - core_width) / 2

s_offset = clad_width / 2 - clad_margin / 2 - y_out
s_length = 5 * s_offset
s_bend1 = pf.Path((length, y_out), core_width).s_bend((s_length, s_offset), relative=True)
s_bend2 = pf.Path((length, -y_out), core_width).s_bend((s_length, -s_offset), relative=True)

y_splitter = pf.Component("Y_SPLITTER")
y_splitter.add("WG_CORE", y_polygon, s_bend1, s_bend2)

In [10]:
clad = pf.envelope(
    y_splitter.get_structures("WG_CORE"), clad_margin, trim_x_min=True, trim_x_max=True
)
y_splitter.add("WG_CLAD", clad)

In [11]:
candidates = y_splitter.detect_ports(["Strip"])
candidates

[Port(center=(0, 0), input_direction=0, spec=PortSpec(description="Strip waveguide", width=2.25, limits=(-1, 1.22), num_modes=1, added_solver_modes=0, polarization="", target_neff=4, default_radius=0, path_profiles=[(0.5, 0, (2, 0)), (2.5, 0, (1, 0))]), extended=True, inverted=False, bend_radius=0),
 Port(center=(4, -0.75), input_direction=180, spec=PortSpec(description="Strip waveguide", width=2.25, limits=(-1, 1.22), num_modes=1, added_solver_modes=0, polarization="", target_neff=4, default_radius=0, path_profiles=[(0.5, 0, (2, 0)), (2.5, 0, (1, 0))]), extended=True, inverted=False, bend_radius=0),
 Port(center=(4, 0.75), input_direction=180, spec=PortSpec(description="Strip waveguide", width=2.25, limits=(-1, 1.22), num_modes=1, added_solver_modes=0, polarization="", target_neff=4, default_radius=0, path_profiles=[(0.5, 0, (2, 0)), (2.5, 0, (1, 0))]), extended=True, inverted=False, bend_radius=0)]

In [12]:
y_splitter.add_port(candidates)
y_splitter
viewer(y_splitter)

In [ ]:
tidy3d_model = pf.Tidy3DModel(port_symmetries=[("P0", "P2", "P1")])
y_splitter.add_model(tidy3d_model, "Tidy3D")

y_splitter.active_model

In [ ]:
lda = np.linspace(1.5, 1.6, 11)
freqs = pf.C_0 / lda

s_matrix = y_splitter.s_matrix(freqs)

In [ ]:
print(s_matrix.elements.keys())

In [ ]:
s_matrix[("P0@0", "P1@0")]

In [ ]:
fig, ax = pf.plot_s_matrix(s_matrix, input_ports=["P0", "P1"])